# Tutorial 2: Working with Language Models in LangChain

In this tutorial, we'll explore how to work with language models in LangChain, focusing on the Groq LLM. We'll cover connecting to the model, creating prompt templates, building chains, and handling responses.

## 1. Connecting to Language Models

First, let's set up our environment and connect to the Groq LLM:

In [31]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

load_dotenv()

# Set your Groq API key
os.environ["GROQ_API_KEY"]

# Initialize the Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.1,
        model_kwargs={"top_p": 0.2, "seed": 1337}
    )

# Test the connection
response = llm.invoke("Hello, world!")
print(response)

content='Hello. How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 39, 'total_tokens': 49, 'completion_time': 0.04, 'prompt_time': 0.009920069, 'queue_time': 0.090482385, 'total_time': 0.049920069}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs': None} id='run-fc9ba970-7365-4097-8627-22991f55302f-0'


## 2. Creating Prompt Templates

Prompt templates allow us to create reusable prompts with input variables:

In [13]:
# Define a simple prompt template
template = """Answer the following question:
You answers in the {language} language.
Question: {question}
Answer: Let's approach this step-by-step:"""


prompt = PromptTemplate(template=template, input_variables=["question","language"])

# Use the prompt template
question = "What is the capital of France?"
formatted_prompt = prompt.format(question=question,language="English")
print(formatted_prompt)

Answer the following question:
You answers in the English language.
Question: What is the capital of France?
Answer: Let's approach this step-by-step:


## 3. Building Simple Prompt Chains

Now, let's create a chain that combines our prompt template with the language model:

In [20]:
# Create a chain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain
result = chain.invoke({"question":"What is the speed of light?","language":"French"})
print(result["text"])

La vitesse de la lumière est une constante physique fondamentale qui représente la vitesse à laquelle la lumière se propage dans le vide. Pour répondre à cette question, nous allons procéder étape par étape.

Étape 1 : La vitesse de la lumière est une constante universelle qui a été mesurée et calculée avec une grande précision.

Étape 2 : Selon la théorie de la relativité restreinte d'Albert Einstein, la vitesse de la lumière est la même pour tous les observateurs, quelle que soit leur vitesse relative.

Étape 3 : La vitesse de la lumière a été mesurée à l'aide de différentes méthodes expérimentales, notamment en utilisant des lasers et des miroirs.

Étape 4 : La vitesse de la lumière a été définie comme une constante physique fondamentale par le Système international d'unités (SI) en 1983.

Étape 5 : La valeur de la vitesse de la lumière est de 299 792 458 mètres par seconde (m/s).

En résumé, la vitesse de la lumière est de 299 792 458 m/s.


## 4. Handling Model Responses

Let's explore different ways to handle and process model responses:

In [26]:
# # Get the raw response
# raw_response = llm.invoke("List three prime numbers.")
# print("Raw response:", raw_response)

# # Using the chain with a dictionary input
# chain_response = chain({"item": "tree Names","language": "French"})
# print("\nChain response:", chain_response['text'])

# Parsing structured output


from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

list_prompt = PromptTemplate(
    template="List 100 {item}. {format_instructions} write only the colors, nothing else",
    input_variables=["item"],
    partial_variables={"format_instructions": format_instructions}
)

chain = LLMChain(llm=llm, prompt=list_prompt)
result = chain.run(item="colors")
parsed_result = output_parser.parse(result)
print("\nParsed list:", parsed_result)


Parsed list: ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet', 'pink', 'brown', 'grey', 'black', 'white', 'purple', 'turquoise', 'silver', 'gold', 'copper', 'bronze', 'beige', 'cream', 'peach', 'coral', 'salmon', 'lavender', 'lilac', 'magenta', 'cyan', 'teal', 'mint', 'aqua', 'navy', 'charcoal', 'ivory', 'champagne', 'plum', 'burgundy', 'garnet', 'ruby', 'emerald', 'sapphire', 'amethyst', 'peridot', 'topaz', 'amber', 'honey', 'mustard', 'sienna', 'umber', 'sepia', 'mahogany', 'walnut', 'ebony', 'onyx', 'jade', 'olive', 'sage', 'moss', 'fern', 'spruce', 'pine', 'cedar', 'sand', 'clay', 'terracotta', 'sienna', 'carmine', 'crimson', 'scarlet', 'vermilion', 'cerulean', 'cobalt', 'azure', 'steel', 'pewter', 'rose', 'blush', 'powder', 'mauve', 'wisteria', 'iris', 'lapis', 'opal', 'pearl', 'taupe', 'mocha', 'caramel', 'tawny', 'goldenrod', 'lemon', 'lime', 'chartreuse', 'celadon', 'seafoam', 'mist', 'fog', 'smoke', 'ash', 'slate', 'granite', 'basalt', 'obsidian', 'coal', 'ink'

## 5. Best Practices for Prompt Engineering

Here are some best practices for effective prompt engineering:

In [30]:
# 1. Be specific and provide context
specific_prompt = PromptTemplate(
    template="You are an expert in {field}. Explain {concept} in simple terms for a beginner.",
    input_variables=["field", "concept"]
)

# 2. Use examples (few-shot learning)
few_shot_prompt = PromptTemplate(
    template="""Classify the sentiment of the following text as positive, negative, or neutral.

Example 1:
Text: I love this product!
Sentiment: Positive

Example 2:
Text: This is the worst experience ever.
Sentiment: Negative

Example 3:
Text: The weather is cloudy today.
Sentiment: Neutral

Now, classify the following text:
Text: {text}
Sentiment:""",
    input_variables=["text"]
)

# 3. Break complex tasks into steps
step_prompt = PromptTemplate(
    template="""To solve the problem '{problem}', follow these steps:
1. Identify the key information
2. Determine the appropriate formula or method
3. Apply the formula or method step-by-step
4. Check your answer

Now, solve the problem:""",
    input_variables=["problem"]
)

# Test the prompts
chains = {
    "Specific": LLMChain(llm=llm, prompt=specific_prompt),
    "Few-shot": LLMChain(llm=llm, prompt=few_shot_prompt),
    "Step-by-step": LLMChain(llm=llm, prompt=step_prompt)
}

for name, chain in chains.items():
    print(f"\n{name} Prompt Result:")
    if name == "Specific":
        chain.run(field="physics", concept="quantum entanglement")
    elif name == "Few-shot":
        chain.run(text="This movie was okay, I guess.")
    else:
        print(chain.run(problem="Calculate the area of a circle with radius 5 cm"))


Specific Prompt Result:

Few-shot Prompt Result:

Step-by-step Prompt Result:
To solve the problem 'Calculate the area of a circle with radius 5 cm', I will follow the given steps:

1. Identify the key information:
The key information in this problem is the radius of the circle, which is 5 cm.

2. Determine the appropriate formula or method:
The formula to calculate the area of a circle is A = πr², where A is the area and r is the radius of the circle.

3. Apply the formula or method step-by-step:
Using the formula A = πr², we can substitute the value of the radius (r = 5 cm) into the formula:
A = π(5)²
A = π(25)
A = 3.14159... * 25
A ≈ 78.54

4. Check your answer:
The calculated area of the circle is approximately 78.54 square centimeters. This answer seems reasonable, as the area of a circle with a radius of 5 cm should be greater than the area of a square with a side length of 5 cm (which is 25 square centimeters).


## Conclusion

In this tutorial, we've explored various aspects of working with language models in LangChain, including connecting to models, creating prompt templates, building chains, handling responses, and implementing best practices for prompt engineering. These skills will serve as a foundation for building more complex applications with LangChain in future tutorials.